!pip install langchain langchain-community langchain-core faiss-cpu tiktoken upstage upstage-ai


In [5]:
import logging
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, TokenTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.passthrough import RunnablePassthrough
import tiktoken
from langsmith import LangSmithClient

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


ImportError: cannot import name 'LangSmithClient' from 'langsmith' (c:\Users\joon\miniconda3\envs\rag\lib\site-packages\langsmith\__init__.py)

In [ ]:
file_path = "all you need is attention.pdf"  # PDF 또는 TXT 파일

if file_path.endswith(".pdf"):
    loader = PyPDFLoader(file_path)
elif file_path.endswith(".txt"):
    loader = TextLoader(file_path)
else:
    raise ValueError("지원되지 않는 파일 형식")

documents = loader.load()
logging.info(f"✅ 문서 로드 완료: {len(documents)} 개")


In [ ]:
splitter_method = "recursive"  # "recursive" 또는 "token"

if splitter_method == "recursive":
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
elif splitter_method == "token":
    splitter = TokenTextSplitter(chunk_size=200, chunk_overlap=50)
else:
    raise ValueError("지원되지 않는 splitter 방식")

split_docs = splitter.split_documents(documents)
logging.info(f"✅ 문서 분할 완료: {len(split_docs)} 개 청크 생성")


In [ ]:
embedding_method = "openai"  # "openai", "bge", "upstage"

if embedding_method == "openai":
    embedding_model = OpenAIEmbeddings()
elif embedding_method == "bge":
    embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en")
elif embedding_method == "upstage":
    embedding_model = HuggingFaceEmbeddings(model_name="Upstage/Solar-embedding")
else:
    raise ValueError("지원되지 않는 Embedding 방식")

logging.info(f"✅ 임베딩 모델 선택: {embedding_method.upper()}")


In [ ]:
vector_db = FAISS.from_documents(split_docs, embedding_model)
logging.info(f"✅ 벡터 DB 구축 완료")


In [ ]:
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})  
# search_type: "similarity", "mmr" (최대 마진 검색)
# k: 반환할 문서 개수

logging.info(f"✅ 검색기 설정 완료 (Top-K: {retriever.search_kwargs['k']}, 방식: {retriever.search_type})")


In [ ]:
llm_model = "openai"  # "openai", "upstage", "lmstudio"

if llm_model == "openai":
    llm = ChatOpenAI(model_name="gpt-4")
elif llm_model == "upstage":
    llm = ChatOpenAI(model_name="solar-10.7b")
elif llm_model == "lmstudio":
    llm = ChatOpenAI(base_url="http://localhost:1234/v1", model_name="lmstudio")
else:
    raise ValueError("지원되지 않는 LLM 모델")

logging.info(f"✅ LLM 모델 선택: {llm_model.upper()}")


In [ ]:
prompt = hub.pull("rlm/rag-prompt")

# 체인 구성
chain = (
    {
        "context": retriever.search_and_merge,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

logging.info("✅ 검색 체인 생성 완료")


In [ ]:
class TokenMonitor:
    """LLM 응답의 토큰 길이를 추적하고 LangSmith에 저장"""

    def __init__(self):
        self.client = LangSmithClient(api_key="your-langsmith-api-key")

    def track(self, response):
        enc = tiktoken.get_encoding("cl100k_base")
        token_count = len(enc.encode(response))
        logging.info(f"✅ 생성된 응답 토큰 수: {token_count}")
        self.client.log_event("response_token", {"token_count": token_count})

token_monitor = TokenMonitor()


In [ ]:
query = "LangChain이란?"
response = chain.invoke({"question": query})
token_monitor.track(response)

print("🔹 AI 응답:", response)
